In [1]:
# import csv
# file = open("WHO_FAQ.csv")
# csvreader = csv.reader(file)
# header = next(csvreader)
# print(header)
# rows = []
# for row in csvreader:
#     rows.append(row)
# print(rows)
# file.close()

In [2]:
# for i in rows:
#   print('{"pats":["'+str(i[0])+'"],\n"replies":["'+str(i[1]) +'"]},\n')

In [181]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [182]:
import json 
import numpy as np 
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [183]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [184]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [185]:
punc_list=['.','/','"','-','?',',',"'"]
covid_vars=["COVID", "Corona", "Coronavirus", "COVID-19"]
exclude_list=["not","nor","don't","what","how","are","you"]
stop_words = [i for i in stopwords.words('english') if (i not in exclude_list)]

def preprocess(example_sent):
  example_sent=example_sent.replace("covid-19","covid").replace("coronavirus","covid").replace("corona","covid").replace("covid19","covid").replace("virus","covid")
  word_tokens = word_tokenize(example_sent)
  filtered_sentence = [stemmer.stem(w) for w in word_tokens if not w.lower() in stop_words]
  filtered_sentence = []
  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)
  for i in punc_list:
    if i in filtered_sentence:
      filtered_sentence.remove(i)
  return ' '.join(filtered_sentence)

In [186]:
with open('intents.json') as file:
    data = json.load(file)
    
train_x = []
train_y = []
bag = []
replies = []


for intent in data['intents']:
    for pat in intent['patterns']:
        s=preprocess(pat)
        train_x.append(s.lower())
        print(s.lower())
        train_y.append(intent['tag'].lower())
    replies.append([i.lower() for i in intent['responses']])
    
    if intent['tag'] not in bag:
        bag.append(intent['tag'].lower())
        
num_classes = len(bag)

hi
hey
is anyone
hello
hay
sup
how are you
how are you
are you fine
how you
how life
how you
bye
see you later
goodbye
thanks
thank you
that 's helpful
thanks help
okay
ok
who are you
what are you
who
what name
what i call you
whats name
your name
could you help
give hand please
can you help
what you
i need support
i need help
support please
what covid
what covid
what covid
what covid
what pandemic
what pandemic
what covid
what covid
what covid
what covid
what covid
what are symptoms covid
symptoms covid
what happens covid
what happens you get covid
what are symptoms covid
get fever covid
what happens i get covid
what get covid
what are symptoms covid
what are symptoms covid
fever symptom covid
cough symptom covid
do old people get covid faster
how covid rise
how covid increase
how people catch covid
how peole get covid
how covid spread
how covid spread
covid spread
how people catch covid
how covid spread
can covid spread air
can covid move air
covid spread air
can covid spread coughin

In [187]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(train_y)
train_y = lbl_encoder.transform(train_y)

In [188]:
max_len = 50
oov_token = "<OOV>"
vocab_size = 10000
embedding_dim = 16

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_x)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_x)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [193]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(layers.LSTM(128))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 50, 16)            160000    
                                                                 
 lstm_5 (LSTM)               (None, 128)               74240     
                                                                 
 dense_34 (Dense)            (None, 40)                5160      
                                                                 
Total params: 239,400
Trainable params: 239,400
Non-trainable params: 0
_________________________________________________________________


In [194]:
print(train_y)

[12 12 12 12 12 12  4  4  4  4  4  4  7  7  7 36 36 36 36 36 36  0  0  0
 18 18 18 18 13 13 13 13 13 13 13  9  9  9  9  9  9  9  9  9  9  9 35 35
 35 35 35 35 35 35 35 20 20 20 20 33 33 33 33 33 33 33 33 33 33 33 33 33
 33 33 33 33 33 33 33 33 35 35 35 35 35 35 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 29 29 29 29 29 29 29  8  8  8  8  8  8  8  8 27 27 27 27 27 27
 27 27 27 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28
 28 28 28 28 28 28 28 28 28 28  3  3  3  3  3  3  3  3  3  3  3 17 17 17
 17 17 17 17 17 17 17 37 37 37 37 37 17 17 17 17 17 17 17 17 17 30 30 30
 30 30 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 25 25 25 25 25 25 25  2  2  2  2  2  2  2  2  2  2  2  2  2
 22 22 22 22 22 22 22 34 34 34 34 34 34 34 34 34 34 21 21 21 21 21 21 19
 19 19 19 19 19 19 32 32 32 32 32 10 10 10 10 10 10 10 10  1  1  1  1  1
  1  1  1 23 23 23 23 23 23 23 23 23 23 23 23 23 23

In [195]:
epochs = 150
history = model.fit(padded_sequences, np.array(train_y), epochs=epochs)

Epoch 1/150
13/13 [==============================] - 3s 89ms/step - loss: 3.6607 - accuracy: 0.0687
Epoch 2/150
13/13 [==============================] - 1s 88ms/step - loss: 3.4962 - accuracy: 0.0941
Epoch 3/150
13/13 [==============================] - 1s 87ms/step - loss: 3.4612 - accuracy: 0.0941
Epoch 4/150
13/13 [==============================] - 1s 88ms/step - loss: 3.4498 - accuracy: 0.0941
Epoch 5/150
13/13 [==============================] - 1s 89ms/step - loss: 3.4324 - accuracy: 0.0941
Epoch 6/150
13/13 [==============================] - 1s 91ms/step - loss: 3.4336 - accuracy: 0.0636
Epoch 7/150
13/13 [==============================] - 1s 89ms/step - loss: 3.4233 - accuracy: 0.0941
Epoch 8/150
13/13 [==============================] - 1s 88ms/step - loss: 3.4090 - accuracy: 0.0941
Epoch 9/150
13/13 [==============================] - 1s 89ms/step - loss: 3.3972 - accuracy: 0.0941
Epoch 10/150
13/13 [==============================] - 1s 90ms/step - loss: 3.3727 - accuracy: 0.1323

In [200]:
model.save("lstm_covid_chatbot_model")


import pickle

# to save the fitted tokenizer
with open('tokenizer_lstm.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder_lstm.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: lstm_covid_chatbot_model/assets


INFO:tensorflow:Assets written to: lstm_covid_chatbot_model/assets


In [201]:
!zip -r lstm_chatmodel.zip lstm_covid_chatbot_model

  adding: lstm_covid_chatbot_model/ (stored 0%)
  adding: lstm_covid_chatbot_model/variables/ (stored 0%)
  adding: lstm_covid_chatbot_model/variables/variables.index (deflated 61%)
  adding: lstm_covid_chatbot_model/variables/variables.data-00000-of-00001 (deflated 47%)
  adding: lstm_covid_chatbot_model/assets/ (stored 0%)
  adding: lstm_covid_chatbot_model/saved_model.pb (deflated 90%)
  adding: lstm_covid_chatbot_model/keras_metadata.pb (deflated 86%)


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [114]:
!pip install colorama

In [196]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    # model = keras.models.load_model('covid_chatbot_model9')

    # # load tokenizer object
    # with open('tokenizer9.pickle', 'rb') as handle:
    #     tokenizer = pickle.load(handle)

    # # load label encoder object
    # with open('label_encoder9.pickle', 'rb') as enc:
    #     lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 50
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        s=preprocess(inp)
        inp=s.lower()
        print(tokenizer.texts_to_sequences([inp]))
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag[0].lower():
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(replies))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hey
[[183]]
ChatBot: Hi there
User: How are you doing?
[[3, 5, 14]]
ChatBot: Im fine!
User: What is corona?
[[4, 2]]
ChatBot: Coronaviruses are a large family of viruses which may cause illness in animals or humans. 
ChatBot: covid is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019.
ChatBot: Avoid smoking and eat healthy
User: How to protect myself?
[[3, 10]]
ChatBot: Keep up to date on the latest covid hotspots (cities or local areas where covid is spreading widely).
User: should I wear mask if i am positive
[[6, 32, 9, 27]]
ChatBot: Only wear a mask if you are ill with covid symptoms (especially coughing) or looking after someone who may have covid. Disposable face mask can only be used once.
ChatBot: If you are not ill or looking after someone who is ill then you are wasting a mask. 
ChatBot: There is a

In [197]:
corr=0
tot=0
for x in data['intents']:
  for inp in x['patterns']:
    tot+=1
    s=preprocess(inp)
    inp=s.lower()

    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                          truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])
    if tag[0].lower()==x['tag'].lower():
      corr+=1

In [198]:
print("Correct: {}\nTotal: {}".format(corr,tot))

Correct: 383
Total: 393


In [199]:
print("Accuracy: {}".format(corr*100/tot))

Accuracy: 97.45547073791349
